## 格付けが下位だが、株価が下がっていない銘柄を探す

+ 年初来で見て、格付けがDだが同セクターで十分下がっていないように見える銘柄を探す
+ `financial_health_grade = Fundamentals.financial_health_grade.latest`
    + https://www.quantopian.com/docs/data-reference/morningstar_fundamentals#financial-health-grade
    + Instead of using accounting-based ratios to formulate a measure to reflect the financial health of a firm, we use structural or contingent claim models. Structural models take advantage of both market information and accounting financial information. The firm's equity in such models is viewed as a call option on the value of the firm's assets. If the value of the assets is not sufficient to cover the firm's liabilities (the strike price), default is expected to occur, and the call option expires worthless and the firm is turned over to its creditors. To estimate a distance to default, the value of the firm's liabilities is obtained from the firm's latest balance sheet and incorporated into the model. We then rank the calculated distance to default and award 10% of the universe A's, 20% B's, 40% C's, 20% D's, and 10% F's. Morningstar calculates this figure in-house on a daily basis.
    + デフォルトまでの距離をランク付けし、ユニバースの10%をA、20%をB、40%をC、20%をD、10%をFとします。モーニングスターはこの数値を毎日自社で計算しています。
    

In [ ]:
MORNINGSTAR_INDUSTRY_GROUP_CODES_after_Nov2019 =  {10110: 'Agricultural Inputs',
 10120: 'Building Materials',
 10130: 'Chemicals,Specialty Chemicals',
 10140: 'Lumber & Wood Production,Paper & Paper Products',
 10150: 'Aluminum,Copper,Other Industrial Metals & Mining,Gold,Silver,Other Precious Metals & Mining',
 10160: 'Coking Coal,Steel',
 10200: 'Auto & Truck Dealerships,Auto Manufacturers,Auto Parts,Recreational Vehicles',
 10220: 'Furnishings, Fixtures & Appliances',
 10230: 'Residential Construction',
 10240: 'Textile Manufacturing,Apparel Manufacturing,Footwear & Accessories',
 10250: 'Packaging & Containers',
 10260: 'Personal Services',
 10270: 'Restaurants',
 10280: 'Apparel Retail,Department Stores,Home Improvement Retail,Luxury Goods,Internet Retail,Specialty Retail',
 10290: 'Gambling,Leisure,Lodging,Resorts & Casinos,Travel Services',
 10310: 'Asset Management',
 10320: 'Banks—Diversified,Banks—Regional,Mortgage Finance',
 10330: 'Capital Markets,Financial Data & Stock Exchanges',
 10340: 'Insurance—Life,Insurance—Property & Casualty,Insurance—Reinsurance,Insurance—Specialty,Insurance Brokers,Insurance—Diversified',
 10350: 'Shell Companies,Financial Conglomerates',
 10360: 'Credit Services',
 10410: 'Real Estate—Development,Real Estate Services,Real Estate—Diversified',
 10420: 'REIT—Healthcare Facilities,REIT—Hotel & Motel,REIT—Industrial,REIT—Office,REIT—Residential,REIT—Retail,REIT—Mortgage,REIT—Specialty,REIT—Diversified',
 20510: 'Beverages—Brewers,Beverages—Wineries & Distilleries',
 20520: 'Beverages—Non-Alcoholic',
 20525: 'Confectioners,Farm Products,Household & Personal Products,Packaged Foods',
 20540: 'Education & Training Services',
 20550: 'Discount Stores,Food Distribution,Grocery Stores',
 20560: 'Tobacco',
 20610: 'Biotechnology',
 20620: 'Drug Manufacturers—General,Drug Manufacturers—Specialty & Generic',
 20630: 'Healthcare Plans',
 20645: 'Medical Care Facilities,Pharmaceutical Retailers,Health Information Services',
 20650: 'Medical Devices,Medical Instruments & Supplies',
 20660: 'Diagnostics & Research',
 20670: 'Medical Distribution',
 20710: 'Utilities—Independent Power Producers,Utilities—Renewable',
 20720: 'Utilities—Regulated Water,Utilities—Regulated Electric,Utilities—Regulated Gas,Utilities—Diversified',
 30810: 'Telecom Services',
 30820: 'Advertising Agencies,Publishing,Broadcasting,Entertainment',
 30830: 'Internet Content & Information,Electronic Gaming & Multimedia',
 30910: 'Oil & Gas Drilling,Oil & Gas E&P,Oil & Gas Integrated,Oil & Gas Midstream,Oil & Gas Refining & Marketing,Oil & Gas Equipment & Services',
 30920: 'Thermal Coal,Uranium',
 31010: 'Aerospace & Defense',
 31020: 'Specialty Business Services,Consulting Services,Rental & Leasing Services,Security & Protection Services,Staffing & Employment Services',
 31030: 'Conglomerates',
 31040: 'Engineering & Construction,Infrastructure Operations,Building Products & Equipment',
 31050: 'Farm & Heavy Construction Machinery',
 31060: 'Industrial Distribution',
 31070: 'Business Equipment & Supplies,Specialty Industrial Machinery,Metal Fabrication,Pollution & Treatment Controls,Tools & Accessories,Electrical Equipment & Parts',
 31080: 'Airports & Air Services,Airlines,Railroads,Marine Shipping,Trucking,Integrated Freight & Logistics',
 31090: 'Waste Management',
 31110: 'Information Technology Services,Software—Application,Software—Infrastructure',
 31120: 'Communication Equipment,Computer Hardware,Consumer Electronics,Electronic Components,Electronics & Computer Distribution,Scientific & Technical Instruments',
 31130: 'Semiconductor Equipment & Materials,Semiconductors,Solar'}


## 銘柄探し

In [ ]:
import numpy as np
import pandas as pd 


from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import QTradableStocksUS, StaticAssets

from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.factors import Returns

from quantopian.pipeline.data.morningstar import Fundamentals

def make_pipeline():
    
    base_universe = QTradableStocksUS()
    close_price = USEquityPricing.close.latest
    
    # sector code 
    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    morningstar_industry_code = Fundamentals.morningstar_industry_code.latest
    financial_health_grade = Fundamentals.financial_health_grade.latest
    prev_change = Returns(inputs = [USEquityPricing.close], window_length=2)
    
    
    return Pipeline(
        columns = {
            'close_price': close_price,
            'morningstar_industry_code':morningstar_industry_code, 
            'morningstar_industry_group_code':morningstar_industry_group_code,
            'financial_health_grade':financial_health_grade,
#             'prev_change':prev_change, 
        },
        
        screen=base_universe
    )
 
start_date='2020-02-17'
end_date='2020-10-08'
    
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date=start_date,
    end_date=end_date,
)



## ランキングで確認する。
+ 取得したデータの最終日のランキングを確認
+ A-Fに何社入っているか



In [ ]:
lastdate = pipeline_output.index.get_level_values(0)[-1]
print(lastdate)
df_lastdate = pipeline_output.loc[lastdate]

def get_tickers_by_ranking(df, label):
    df_rank = df[df["financial_health_grade"] == label]
    tickers = df_rank.index
    return tickers 

A = get_tickers_by_ranking(df_lastdate, "A")
B = get_tickers_by_ranking(df_lastdate, "B")
C = get_tickers_by_ranking(df_lastdate, "C")
D = get_tickers_by_ranking(df_lastdate, "D")
F = get_tickers_by_ranking(df_lastdate, "F")
NA = get_tickers_by_ranking(df_lastdate, np.nan) # 格付けされていない銘柄もある

len(A), len(B), len(C), len(D), len(F) 

In [ ]:
A.map(lambda x: x.asset_name)

## 格付けごとに株価推移
+ 上記で使った `pipeline` を使ってもいいが、銘柄によっては この時期に `QTradableStocksUS` に出たり入ったりしている銘柄もあるので、`price` を使う
+ ランキングごとに終値を取得し、前日比の平均を描画する。

In [ ]:
from quantopian.research import prices, symbols


close_price_A = prices(assets=A,start=start_date, end=end_date,)
close_price_B = prices(assets=B,start=start_date, end=end_date,)
close_price_C = prices(assets=C,start=start_date, end=end_date,)
close_price_D = prices(assets=D,start=start_date, end=end_date,)
close_price_F = prices(assets=F,start=start_date, end=end_date,)
close_price_NA = prices(assets=NA,start=start_date, end=end_date,)
close_price_SPY = prices(assets="SPY",start=start_date, end=end_date,)


In [ ]:
close_price_NA.head(3)

In [ ]:
close_price_A.pct_change().mean(axis=1).cumsum().plot(label="A", legend=True)
close_price_B.pct_change().mean(axis=1).cumsum().plot(label="B", legend=True)
close_price_C.pct_change().mean(axis=1).cumsum().plot(label="C", legend=True)
close_price_D.pct_change().mean(axis=1).cumsum().plot(label="D", legend=True)
close_price_F.pct_change().mean(axis=1).cumsum().plot(label="F", legend=True)
close_price_NA.pct_change().mean(axis=1).cumsum().plot(label="NA", legend=True)
close_price_SPY.pct_change().cumsum().plot(label="SPY", legend=True, linewidth=3, color='#e4007f')



In [ ]:
close_price_A.pct_change().mean(axis=1).sum()

In [ ]:
close_price_F.pct_change().mean(axis=1).sum()

In [ ]:
close_price_F.pct_change().dropna().describe().T

In [ ]:
close_price_F.drop(symbols("WLL"), axis=1).pct_change().cumsum().dropna().plot()#.describe().T

## セクターごとにカウント

+ セクターごとにランキングの比率を確認
+ セクター構成銘柄数のバラツキが大きすぎるので、数が大きいセクタに関しては何かしらの事が言えるかもしれない

In [ ]:
#sector_count = df_lastdate.groupby("morningstar_industry_group_code")["morningstar_industry_code"].count()
#
#df_lastdate.groupby(["financial_health_grade","morningstar_industry_code" ])["close_price"].count().xs("A", level=0)

by_sector = df_lastdate.groupby(["morningstar_industry_group_code" ])["close_price"].count()
by_sector_ranking = df_lastdate.groupby(["financial_health_grade","morningstar_industry_group_code" ])["close_price"].count()

sector_A = by_sector_ranking.xs("A", level=0)
sector_B = by_sector_ranking.xs("B", level=0)
sector_C = by_sector_ranking.xs("C", level=0)
sector_D = by_sector_ranking.xs("D", level=0)
sector_F = by_sector_ranking.xs("F", level=0)
sector_NA = by_sector_ranking.xs(np.nan, level=0)

df_by_sector = pd.DataFrame({"A" : sector_A / by_sector,
              "B" : sector_B / by_sector,
              "C" : sector_C / by_sector,
              "D" : sector_D / by_sector,
              "F" : sector_F / by_sector,
              "NA" : sector_NA / by_sector,
             })
              
# 当該セクターに入っている銘柄数が多い順に並び替え
df_by_sector.loc[by_sector.sort_values(ascending=False).index]


In [ ]:
df_by_sector.sort_values(by="D", ascending=False).head(10)

## 10320（Bank）でDランクの銘柄で他より戻っている銘柄を探す


In [ ]:
def get_price_by_rank_and_sector(df_origin, rank, sector_number, start, end):
    
    symbols = df_origin[(df_origin["financial_health_grade"] == rank) & 
                        (df_origin["morningstar_industry_group_code"] == sector_number)].index
    
    return  prices(assets=symbols,start=start, end=end,)

df_Bank_D = get_price_by_rank_and_sector(df_lastdate, "D", 31080, start_date,end_date)
df_Bank_D

In [ ]:
df_Bank_D.pct_change().dropna().describe().T.sort_values(by="mean", ascending=False).head()

In [ ]:
stocks = df_Bank_D.pct_change().dropna().describe().T.sort_values(by="mean", ascending=False).head(5).index
df_Bank_D[stocks].pct_change().cumsum().plot()

In [ ]:
stocks = df_Bank_D.pct_change().dropna().describe().T.sort_values(by="mean", ascending=True).head(10).index
df_Bank_D[stocks].pct_change().cumsum().plot()


## セクターとして第二波のあとに戻りそうなセクター、死にそうなセクター

+ Dに入っている中で、Restaurantなどは第二波がくると本当に厳しそう
+ Dに入っている中で、第二波後もう一度大きくのびそう
+ つまり、一波目のあと、（3/23）にSPと比べてどのくらい戻ったかを確認

In [ ]:
# まずは全体を見る
benchmark = close_price_SPY.apply(np.log).diff()

df_rank_compare_sp = pd.DataFrame({
    "A":close_price_A.apply(np.log).diff().mean(axis=1) - benchmark,
    "B":close_price_B.apply(np.log).diff().mean(axis=1) - benchmark,
    "C":close_price_C.apply(np.log).diff().mean(axis=1) - benchmark,
    "D":close_price_D.apply(np.log).diff().mean(axis=1) - benchmark,
    "F":close_price_F.apply(np.log).diff().mean(axis=1) - benchmark,})

df_rank_compare_sp.cumsum().plot()



In [ ]:
def make_pipeline_for_symbols(symbols):
    
    base_universe = StaticAssets(symbols)
    close_price = USEquityPricing.close.latest
    
    # sector code 
    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    morningstar_industry_code = Fundamentals.morningstar_industry_code.latest
    financial_health_grade = Fundamentals.financial_health_grade.latest
    
    return Pipeline(
        columns = {
            'close_price': close_price,
            'morningstar_industry_code':morningstar_industry_code, 
            'morningstar_industry_group_code':morningstar_industry_group_code,
            'financial_health_grade':financial_health_grade,
        },
        
        screen=base_universe
    )
 
pipeline_output_D = run_pipeline(
    make_pipeline_for_symbols(D),
    start_date=start_date,
    end_date=end_date
)


for key, grp in list(pipeline_output_D.groupby("morningstar_industry_group_code")["close_price"]):
    if key != -1:
        grp.unstack().apply(np.log).diff().cumsum().plot(
            title="{} {}".format(key,  MORNINGSTAR_INDUSTRY_GROUP_CODES_after_Nov2019[key]))

# list(pipeline_output_D.groupby("morningstar_industry_group_code")["close_price"])[0][1].unstack()